In [ ]:
#IMPORTAR AS BIBLIOTECAS
from selenium import webdriver #controlar o navegador
from selenium.webdriver.chrome.service import Service #Gerenciar servico do CrhomeDriver
from selenium.webdriver.chrome.options import Options #Config do Navegador
from webdriver_manager.chrome import ChromeDriverManager #Gerenciar intalacao do CrhomeDriver
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup #Processar html da pagina
import math
import re
import time
import pandas as pd


In [ ]:
# Iniciando sessao do Navegador -  Selenium WebDriver(JS BOT)
session_options = Options() 

In [ ]:
# Passando argumentos de inicialicaczao (Parametros)
arguments = [
    '--lang=pt-BR', '--disable-notifications', "--headless"
    ]

for arg in arguments:
    session_options.add_argument(arg)

In [ ]:
# Carregar o Navegador (Drivers)
navigator = webdriver.Chrome( options=session_options)
# Ver se consegue usar opcoes basicas
#options = get_default_chrome_options()
    #driver = webdriver.Chrome(options=options)

In [ ]:
#Correcao de espera
errors = [NoSuchElementException, ElementNotInteractableException,ElementNotVisibleException]
wait = WebDriverWait(navigator, timeout=2, poll_frequency=.2, ignored_exceptions=errors)

In [ ]:
#URL de busca
url = "https://www.kabum.com.br/hardware/processadores/processador-intel"
navigator.get(url)


In [ ]:
# Paginacao
qtd_products = wait.until(expected_conditions.element_to_be_clickable((By.ID, "listingCount")))
qtd_products = qtd_products.text
numbers = re.findall(r'\d',qtd_products)
qtd = int(''.join(numbers))
last_page = math.ceil(qtd/20)



In [ ]:
# Dicionario de produtos
dict_produtos = {"Descricao":[], "Preco":[]}


In [ ]:
# Percorrendo pagina com espera
for i in range(1,last_page+1):
    url_pag = f"https://www.kabum.com.br/hardware/processadores/processador-intel?page_number={i}"
    navigator.get(url_pag)
    #Tratamento de espera
    products = wait.until(expected_conditions.visibility_of_all_elements_located((By.CLASS_NAME,"productCard")))

In [ ]:
# Buscando informacoes de produtos
 #navigator.implicitly_wait(10)
 for product in products:
        navigator.execute_script("arguments[0].scrollIntoView();", product) 
        product_name = product.find_element(By.CLASS_NAME, "nameCard").text
        product_price = product.find_element(By.CLASS_NAME, "priceCard").text
        #Adiciona na lista descricao e preco
        dict_produtos["Descricao"].append(product_name)
        dict_produtos["Preco"].append(product_price)

In [ ]:
# Cria arquivo csv
df = pd.DataFrame(dict_produtos)
df.to_csv("Processadores_Intel.csv", encoding="utf-8", sep=";")

print("Fim! Arquivo criado com sucesso.")